In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
# class EncoderRNN(nn.Module):
#
#     def __init__(self, input_size, hidden_size):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding(input_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden

SyntaxError: invalid syntax (4290617539.py, line 1)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.sofmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

In [ ]:
class AttDecoderRNN(nn.Module):

    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=512):
        super(AttDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)

        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)

        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)


    def forward(self, input, hidden, encoder_output):
        embedded = self.embedding(self.output_size, self.hidden_size).view(1, 1, -1)
        embedded = self.dropout(embedded)

        # Attention Part
        attn_weight = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1
        )
        attn_applied = torch.bmm(attn_weight.unsqueeze(0), encoder_output.unsqueeze(0))

        # Output Part
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)

        return output, hidden, attn_weight

In [ ]:
class MyRNN():
    pass

In [ ]:
class MyLoss(nn.Module):
    def __init__(self):
        super(MyLoss, self).__init__()

    def forward(self, x, y):
        temp = torch.pow((x-y), 2)
        return torch.mean(temp)

In [9]:
class L2_Loss(nn.Module):
    def __init__(self):
        super(L2_Loss, self).__init__()
    def forward(self, x, y):
        return torch.mean(torch.pow((x-y), 2))


In [ ]:
class L1_Loss(nn.Module):
    def __init__(self):
        super(L1_Loss, self).__init__()

    def forward(self, x, y):
        return torch.mean(torch.abs(x-y))

In [ ]:
def softmax(x):
    x_exp = torch.exp(x)
    partition = torch.sum(x_exp)
    return x_exp/partition

In [10]:
class SoftMax(nn.Module):
    def __init__(self):
        super(SoftMax, self).__init__()

    def forward(self, x):
        x_exp = torch.exp(x)
        partition = torch.sum(x_exp)
        return x_exp/partition

In [16]:
t = torch.tensor([-0.3623, -0.6115,  0.7283,  0.4699,  2.3261,  0.1599])
print(t)
print(t.shape)

tensor([-0.3623, -0.6115,  0.7283,  0.4699,  2.3261,  0.1599])
torch.Size([6])


In [17]:
result = t.view(-1,1)
print(result)
print(result.shape)

tensor([[-0.3623],
        [-0.6115],
        [ 0.7283],
        [ 0.4699],
        [ 2.3261],
        [ 0.1599]])
torch.Size([6, 1])


In [18]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat.gather(1, y.view(-1, 1)))

In [19]:
def accuracy(y, y_hat):
    prediction = torch.argmax(y_hat, dim=1)
    mask = prediction==y
    return mask.float().mean().item()

In [20]:
def accuracy(y, y_hat):
    prediction = torch.argmax(y_hat, dim=1)
    result = prediction==y
    return torch.sum(result.float()).item()

In [ ]:
def L1_Loss(y, y_hat):
    temp = torch.mean(torch.abs(y-y_hat))
    return temp


In [ ]:
def L2_Loss(y, y_hat):
    result = torch.pow((y-y_hat), 2)
    return torch.mean(result)

In [21]:
def softmax(x):
    x_exp = torch.exp(x)
    partition = torch.sum(x_exp)
    return x_exp/partition